In [0]:
%%capture

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from collections import Counter,OrderedDict
import math
from tabulate import tabulate
import heapq as hp

nltk.download('stopwords')

In [0]:
data = pd.read_csv('https://raw.githubusercontent.com/dnrocha1/information_retrieval/master/lab02/data/results.csv')

In [0]:
#pre-process

stopwords = stopwords.words('portuguese')
documents = data['text'].apply(lambda x: x.lower())

regex = RegexpTokenizer(r'\b[A-zÀ-ú-\'\d]{3,}')
# tokens = regex.tokenize(texts)

In [0]:
doc_list = []

def build_index(documents=documents):
  inverted_list = {}

  n_doc = 0
  for document in documents:
    n_doc += 1
    doc_list.append(n_doc)
    token = [w for w in regex.tokenize(document) if w not in stopwords]
    counter = list(Counter(token).items())
    for elem in counter:
      key = elem[0]
      freq = elem[1]
      if key in inverted_list.keys():
        if n_doc not in inverted_list[key][0]:
          inverted_list[key].append((n_doc,freq))
      else:
        inverted_list[key] = [(n_doc,freq)]
  
  return inverted_list

index = build_index()

## Questão 1

In [0]:
%%capture
def mutual_info(word,index=index):
  
  ranking = {}
  docs_word = set(map(lambda x: x[0], index[word]))
  n_a = len(docs_word)
  
  for key in index:
    if key != word:
      docs_b = set(map(lambda x: x[0], index[key]))
      n_b = len(docs_b)
      n_ab = len(docs_word & docs_b)
      if n_ab == 0: continue
      ranking[key] = n_ab/(n_a*n_b)
  
  return OrderedDict(sorted(ranking.items(), key=lambda x: x[1], reverse=True))
  
mutual_info("governo")

In [0]:
%%capture
def expected_mutual_info(word,index=index):
  
  ranking = {}
  docs_word = set(map(lambda x: x[0], index[word]))
  n_a = len(docs_word)
  n = len(documents)
  
  for key in index:
    if key != word:
      docs_b = set(map(lambda x: x[0], index[key]))
      n_b = len(docs_b)
      n_ab = len(docs_word & docs_b)
      if n_ab == 0: continue
      ranking[key] = n_ab * math.log(n*(n_ab/(n_a*n_b)))
  
  return OrderedDict(sorted(ranking.items(), key=lambda x: x[1], reverse=True))
  
expected_mutual_info("bolsonaro")

In [0]:
%%capture
def chi_square(word,index=index):
  
  ranking = {}
  docs_word = set(map(lambda x: x[0], index[word]))
  n_a = len(docs_word)
  n = len(documents)
  
  for key in index:
    if key != word:
      docs_b = set(map(lambda x: x[0], index[key]))
      n_b = len(docs_b)
      n_ab = len(docs_word & docs_b)
      if n_ab == 0: continue
      ranking[key] = (n_ab-(1/n)*n_a*n_b)**2 / (n_a*n_b)
  
  return OrderedDict(sorted(ranking.items(), key=lambda x: x[1], reverse=True))
  
chi_square("bolsonaro")

In [0]:
%%capture
def dice(word,index=index):
  
  ranking = {}
  docs_word = set(map(lambda x: x[0], index[word]))
  n_a = len(docs_word)
  
  for key in index:
    if key != word:
      docs_b = set(map(lambda x: x[0], index[key]))
      n_b = len(docs_b)
      n_ab = len(docs_word & docs_b)
      if n_ab == 0: continue
      ranking[key] = n_ab/(n_a+n_b)
  
  return OrderedDict(sorted(ranking.items(), key=lambda x: x[1], reverse=True))
  
dice("governo")

In [0]:
# define o conjunto de consultas e a função que constroi a tabela comparativa

queries = ["educação","polícia","cortes","presidente","segurança"]

def build_table(query):
  df = pd.DataFrame()

  df['MIM'] = list(mutual_info(query).keys())
  df['EMIM'] = list(expected_mutual_info(query).keys())
  df['X2'] = list(chi_square(query).keys())
  df['Dice'] = list(dice(query).keys())
  
  return df.head(10)

In [48]:
for query in queries:
  df = build_table(query)
  print(query)
  print(tabulate(df, headers='keys', tablefmt='psql'))
  print("\n")

educação
+----+--------------+----------+-------------+-------------+
|    | MIM          | EMIM     | X2          | Dice        |
|----+--------------+----------+-------------+-------------|
|  0 | enriquecer   | social   | aluno       | escolas     |
|  1 | controversa  | alunos   | vélez       | alunos      |
|  2 | resgatar     | escolas  | alunos      | social      |
|  3 | fabian       | discurso | ex-alunos   | mudanças    |
|  4 | salvioli     | escola   | aula        | discurso    |
|  5 | ouve         | governo  | aulas       | escola      |
|  6 | reverbera    | mudanças | mec         | comissão    |
|  7 | napolitano   | comissão | pasta       | necessidade |
|  8 | negacionismo | vélez    | escolas     | jovens      |
|  9 | distorce     | pasta    | educacional | pasta       |
+----+--------------+----------+-------------+-------------+


polícia
+----+------------+--------------+--------------+--------------+
|    | MIM        | EMIM         | X2           | Dice        

De acordo com os resultados produzidos, é possível observar que o EMIM apresenta palavras mais condizentes com os termos correspondentes. A métrica Dice também possui bons resultados, no entanto as palavras produzidas são mais gerais, ao contrário do EMIM, que ao meu modo de ver, é um pouco mais específico.

## Questão 2

In [0]:
# retorna item composto por score,document

def term_at_time(query, index, k=100):
  query_words = query.split()    
  inverted_lists = []
  acc = {}

  for word in query_words:
    if word in index.keys():
      inverted_lists.append(index[word])

  for lst in inverted_lists:
    for posting in lst:
      d = posting[0]
      freq = posting[1]
      if d in acc.keys():
        acc[d] = acc[d] + freq
      else:
        acc[d] = freq
  
  priority_queue = list(map(lambda elem: (elem[1],elem[0]), acc.items()))
  
  hp._heapify_max(priority_queue)
  
  top_k = []
  
  for i in range(1,k+1):
    if priority_queue != []:
      top = hp._heappop_max(priority_queue)
      top_k.append(top)
  
  return top_k

In [0]:
pd.options.display.max_colwidth = 500

def print_docs(query):
  query_result = term_at_time(query,index,10)
  doc_list = [doc for score,doc in query_result]
  print("Os top 10 documentos processados para \"" + query + "\" foram:", doc_list)
  return doc_list

def print_title(doc_list): 
  return data.loc[list(map(lambda x: x-1, doc_list))][['title','subtitle']]

def get_similar_terms(query, k):
  return query + " " + " ".join(list(expected_mutual_info(query).keys())[:k])

Temos que "educação" é a primeira palavra dentro do conjunto de queries definido. Vamos observar o comportamento desse termo para o processamento dessa consulta. Observaremos os títulos e subtitulos dos documentos.

In [200]:
query = "educação"
docs = print_docs(query)

Os top 10 documentos processados para "educação" foram: [221, 222, 130, 239, 160, 37, 215, 110, 233, 205]


In [201]:
print_title(docs)

,title,subtitle
220,“A gente está vendo a corrosão geral das políticas estabelecidas no MEC”,Para diretor da Ação Educativa pautas reais vêm sendo abandonadas por questões ideológicas.
221,A jovem deputada que jogou contra as cordas o ministro da Educação e sua “lista de desejos”,Neófita no Congresso Tabata Amaral (PDT-SP) sugere que Ricardo Vélez se demita do MEC o deixa em silêncio e agrava ainda mais a situação do ministro olavista de Bolsonaro
129,A universidade que quer oferecer uma educação do nível de Harvard pela metade do preço,Chama-se Minerva e seus alunos estudam em sete cidades espalhadas pelo mundo provocando uma revolução no panorama universitário dos EUA
238,Saúde mental dos estudantes mais um desafio para as escolas brasileiras,Massacre de Suzano reabre debate sobre papel das instituições de ensino em identificar transtornos psiquiátricos. 80% dos alunos da rede pública com algum problema não recebem tratamento
159,Querer colocar a Bíblia no centro da educação escolar significa devolver o Brasil às cavernas,A ideia do novo Governo ultraconservador de Bolsonaro é colocar a Bíblia no centro do ensino não somente como matéria religiosa e sim para servir de base até para matérias como matemática história e geografia
36,Os percalços dos planos A e B de Paulo Guedes para controlar o gasto público,Superministro propõe incluir sistema de capitalização em reforma da Previdência. Ele já pôs na mesa banir regras de gasto obrigatório com saúde e educação
214,Bolsonaro X Maia a trama se adensa enquanto o dólar tem forte alta,Investidores expressaram cautela em meio a mais um dia de trocas de farpas.
109,Só sete jovens negros no Olimpo da educação pública de Nova York,O célebre colégio Stuyvesant se torna também o epicentro da segregação: apenas 7 dos 895 novos alunos para o próximo ano letivo são afro-americanos enquanto três quartos são asiáticos
232,Ministro promete mudar livros didáticos por “visão mais ampla” da ditadura,Em entrevista ao 'Valor' Ricardo Vélez da Educação diz que não ocorreu um golpe em 1964
204,A guerra de todos contra todos no Governo Bolsonaro,Com chamado aberto à comemoração do golpe de 1964 presidente dobra sua aposta na polarização política


A query será expandida para os top-3 termos de acordo com o método usado na questão 1.

In [191]:
q3 = get_similar_terms(query, 3)
docs = print_docs(q3)

Os top 10 documentos processados para "educação social alunos escolas" foram: [221, 239, 130, 110, 222, 214, 203, 248, 160, 215]


In [192]:
print_title(docs)

,title,subtitle
220,“A gente está vendo a corrosão geral das políticas estabelecidas no MEC”,Para diretor da Ação Educativa pautas reais vêm sendo abandonadas por questões ideológicas.
238,Saúde mental dos estudantes mais um desafio para as escolas brasileiras,Massacre de Suzano reabre debate sobre papel das instituições de ensino em identificar transtornos psiquiátricos. 80% dos alunos da rede pública com algum problema não recebem tratamento
129,A universidade que quer oferecer uma educação do nível de Harvard pela metade do preço,Chama-se Minerva e seus alunos estudam em sete cidades espalhadas pelo mundo provocando uma revolução no panorama universitário dos EUA
109,Só sete jovens negros no Olimpo da educação pública de Nova York,O célebre colégio Stuyvesant se torna também o epicentro da segregação: apenas 7 dos 895 novos alunos para o próximo ano letivo são afro-americanos enquanto três quartos são asiáticos
221,A jovem deputada que jogou contra as cordas o ministro da Educação e sua “lista de desejos”,Neófita no Congresso Tabata Amaral (PDT-SP) sugere que Ricardo Vélez se demita do MEC o deixa em silêncio e agrava ainda mais a situação do ministro olavista de Bolsonaro
213,A tensão em escolas e universidades na esteira do massacre de Suzano,Instituições buscam formas para lidar com ameaças várias delas já identificadas pelas autoridades policiais como trote de adolescentes
202,Bolsonaro e o desenvolvimento dos filhos do Bolsa Família: o que os pesquisadores realmente dizem,Presidente afirmou de forma imprecisa que estudo do Governo chegou a conclusão que o desenvolvimento intelectual de crianças do programa equivalia a 1/3 da média mundial
247,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,Para filósofa e matemática campo progressista deve se opor à reforma da Previdência de Bolsonaro apresentando seu próprio projeto. Defende que a renovação deve vir a partir dos municípios
159,Querer colocar a Bíblia no centro da educação escolar significa devolver o Brasil às cavernas,A ideia do novo Governo ultraconservador de Bolsonaro é colocar a Bíblia no centro do ensino não somente como matéria religiosa e sim para servir de base até para matérias como matemática história e geografia
214,Bolsonaro X Maia a trama se adensa enquanto o dólar tem forte alta,Investidores expressaram cautela em meio a mais um dia de trocas de farpas.


Agora, a query expandida para os top-5 termos.

In [193]:
q5 = get_similar_terms(query, 5)
docs = print_docs(q5)

Os top 10 documentos processados para "educação social alunos escolas discurso escola" foram: [239, 221, 130, 214, 110, 222, 203, 160, 115, 37]


In [194]:
print_title(docs)

,title,subtitle
238,Saúde mental dos estudantes mais um desafio para as escolas brasileiras,Massacre de Suzano reabre debate sobre papel das instituições de ensino em identificar transtornos psiquiátricos. 80% dos alunos da rede pública com algum problema não recebem tratamento
220,“A gente está vendo a corrosão geral das políticas estabelecidas no MEC”,Para diretor da Ação Educativa pautas reais vêm sendo abandonadas por questões ideológicas.
129,A universidade que quer oferecer uma educação do nível de Harvard pela metade do preço,Chama-se Minerva e seus alunos estudam em sete cidades espalhadas pelo mundo provocando uma revolução no panorama universitário dos EUA
213,A tensão em escolas e universidades na esteira do massacre de Suzano,Instituições buscam formas para lidar com ameaças várias delas já identificadas pelas autoridades policiais como trote de adolescentes
109,Só sete jovens negros no Olimpo da educação pública de Nova York,O célebre colégio Stuyvesant se torna também o epicentro da segregação: apenas 7 dos 895 novos alunos para o próximo ano letivo são afro-americanos enquanto três quartos são asiáticos
221,A jovem deputada que jogou contra as cordas o ministro da Educação e sua “lista de desejos”,Neófita no Congresso Tabata Amaral (PDT-SP) sugere que Ricardo Vélez se demita do MEC o deixa em silêncio e agrava ainda mais a situação do ministro olavista de Bolsonaro
202,Bolsonaro e o desenvolvimento dos filhos do Bolsa Família: o que os pesquisadores realmente dizem,Presidente afirmou de forma imprecisa que estudo do Governo chegou a conclusão que o desenvolvimento intelectual de crianças do programa equivalia a 1/3 da média mundial
159,Querer colocar a Bíblia no centro da educação escolar significa devolver o Brasil às cavernas,A ideia do novo Governo ultraconservador de Bolsonaro é colocar a Bíblia no centro do ensino não somente como matéria religiosa e sim para servir de base até para matérias como matemática história e geografia
114,Esportes vivem paralisia sem planos detalhados e nomeações travadas,Rebaixada de ministério a secretaria política esportiva de Bolsonaro espelha modelo do regime ditatorial com militares na linha de frente engessada pela falta de autonomia
36,Os percalços dos planos A e B de Paulo Guedes para controlar o gasto público,Superministro propõe incluir sistema de capitalização em reforma da Previdência. Ele já pôs na mesa banir regras de gasto obrigatório com saúde e educação


Por último, query expandida para os top-10 termos.

In [195]:
q10 = get_similar_terms(query, 10)
docs = print_docs(q10)

Os top 10 documentos processados para "educação social alunos escolas discurso escola governo mudanças comissão vélez pasta" foram: [221, 239, 115, 130, 248, 222, 214, 173, 166, 215]


In [196]:
print_title(docs)

,title,subtitle
220,“A gente está vendo a corrosão geral das políticas estabelecidas no MEC”,Para diretor da Ação Educativa pautas reais vêm sendo abandonadas por questões ideológicas.
238,Saúde mental dos estudantes mais um desafio para as escolas brasileiras,Massacre de Suzano reabre debate sobre papel das instituições de ensino em identificar transtornos psiquiátricos. 80% dos alunos da rede pública com algum problema não recebem tratamento
114,Esportes vivem paralisia sem planos detalhados e nomeações travadas,Rebaixada de ministério a secretaria política esportiva de Bolsonaro espelha modelo do regime ditatorial com militares na linha de frente engessada pela falta de autonomia
129,A universidade que quer oferecer uma educação do nível de Harvard pela metade do preço,Chama-se Minerva e seus alunos estudam em sete cidades espalhadas pelo mundo provocando uma revolução no panorama universitário dos EUA
247,Tatiana Roque: “O problema da esquerda não é a pauta dita identitária mas sim a lacração”,Para filósofa e matemática campo progressista deve se opor à reforma da Previdência de Bolsonaro apresentando seu próprio projeto. Defende que a renovação deve vir a partir dos municípios
221,A jovem deputada que jogou contra as cordas o ministro da Educação e sua “lista de desejos”,Neófita no Congresso Tabata Amaral (PDT-SP) sugere que Ricardo Vélez se demita do MEC o deixa em silêncio e agrava ainda mais a situação do ministro olavista de Bolsonaro
213,A tensão em escolas e universidades na esteira do massacre de Suzano,Instituições buscam formas para lidar com ameaças várias delas já identificadas pelas autoridades policiais como trote de adolescentes
172,Não temos tempo a perder. A hora de unir as forças democráticas é agora,Essa articulação requererá deixar de lado diferenças e firmar um compromisso pela defesa do interesse da sociedade brasileira como um todo
165,Bolsonaro manda festejar o crime,Ao determinar a comemoração do golpe militar de 1964 o antipresidente busca manter o ódio ativo e barrar qualquer possibilidade de justiça
214,Bolsonaro X Maia a trama se adensa enquanto o dólar tem forte alta,Investidores expressaram cautela em meio a mais um dia de trocas de farpas.


É possível observar pelos títulos que a medida que realizamos as expansões o conteúdo dos mesmos tornou-se menos generalista, focando-se em conteúdo relacionado à educação, mas com forte viés político. Entretanto, isso é condizente com o termo "governo" é um dos termos que mais aparece em documentos, o que pode explicar essa tendência dos resultados apresentarem essa característica.